<a href="https://colab.research.google.com/github/josiahlin119/IMDB-Final-Project/blob/main/Imdb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:


!pip install -1 tf-model




Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -1


In [60]:
import numpy as np


In [61]:
#Load the libraries


# import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup

import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#deep learning

import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from tensorflow.keras import models
from tensorflow.keras import layers

In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/josiah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
imdb_data=pd.read_csv('https://raw.githubusercontent.com/josiahlin119/IMDB-Final-Project/main/ReviewData/IMDB%20Dataset.csv')
print(imdb_data.shape)
imdb_data.head

(50000, 2)


<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [65]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [66]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [67]:


# Lets encode labels: each label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
imdb_data['sentiment'] = label_encoder.fit_transform(imdb_data['sentiment'])
imdb_data.head()



,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [68]:

tokenizer=ToktokTokenizer()
#filter out all stop words which do not contribute much to the learning
stopword_list = nltk.corpus.stopwords.words('english')


Remove irrelevent symbols

In [69]:
# def strip_html(text):
#   soup = BeautifulSoup(text, "html.parser")
#   return soup.get_text()
# def remove_between_square_brackets(text):
#   return re.sub('\[[^]]*\]','',text)
# def remove_special_characters(text, remove_digits=True):
#   pattern=r'[^a-zA-Z0-9\s]'
#   text=re.sub(pattern,'',text)
#   return text
# def denoise_text(text):
#   text = strip_html(text)
#   text = remove_between_square_brackets(text)
#   text = remove_special_characters(text)
#   return text
# imdb_data['review']=imdb_data['review'].apply(denoise_text)


Word Stemming

In [70]:
# def my_stemmer(text):
#   ps = nltk.porter.PorterStemmer()
#   text = ' '.join([ps.stem(word) for word in text.split()])
#   return text
# imdb_data['review']=imdb_data['review'].apply(my_stemmer)


Remove Stop Words

In [71]:
# stop=set(stopwords.words('english'))
# print(stop)

# def remove_stopwords(text, is_lower_case=False):
#   tokens = tokenizer.tokenize(text)
#   tokens = [token.strip() for token in tokens]
#   if is_lower_case:
#     filtered_tokens = [token for token in tokens if token not in stopword_list]
#   else:
#     filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
#   filtered_text= ' '.join(filtered_tokens)
#   return filtered_text
# imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)



In [72]:
# cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,2))
# cv_train_reviews=cv.fit_transform(train_reviews)
# cv_test_reviews=cv.transform(test_reviews)
# print('BOW_cv_train: ', cv_train_reviews.shape)
# print('BOW_cv_test:', cv_test_reviews.shape)
# all_ngrams = cv.get_feature_names()
# print(all_ngrams)
# num_ngrams = min(50, len(all_ngrams))

In [73]:
# #Tfidf vectorizer
# tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,2))
# #transformed train reviews
# tv_train_reviews=tv.fit_transform(train_reviews)
# #transformed test reviews
# tv_test_reviews=tv.transform(test_reviews)
# print('Tfidf_train:',tv_train_reviews.shape)
# print('Tfidf_test:',tv_test_reviews.shape)



In [74]:


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1, 2)
TOP_K = 20000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

def ngram_vectorize(texts, labels):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ,
    }
    # Learn Vocab from train texts and vectorize train and val sets
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    # Select best k features, with feature importance measured by f_classif
    # Set k as 20000 or (if number of ngrams is less) number of ngrams   
    selector = SelectKBest(f_classif, k=min(TOP_K, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    transformed_texts = selector.transform(transformed_texts).astype('float32')
    return transformed_texts
# Vectorize the data

vect_data = ngram_vectorize(imdb_data['review'],imdb_data['sentiment'])



/Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1796: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


In [75]:
from sklearn.model_selection import train_test_split

X = vect_data.toarray()
print("shape of X_train", X.shape)
input_shape = X.shape[1:]

print("input_shape", input_shape)
y = (np.array(imdb_data['sentiment']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

vect_data.shape

shape of X_train (50000, 20000)
input_shape (20000,)


(50000, 20000)

In [76]:
max_sequence_length = 100
loss_function = 'binary_crossentropy'
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001
)

addition_metrics = ['accuracy']
number_of_epochs=100
verbosity_mode =True
validation_split = 0.2
#add a callback function to speed up the process 
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]


Deep Learning Model: Keras. The input data are transformed tokens insteaed of the original words

In [77]:
# print("type of cv_revie",type(cv_train_reviews))


model = keras.Sequential()
print(y.shape)
model.add(layers.Dropout(rate=0.2, input_shape=input_shape))
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='softmax'))
model.compile(optimizer=optimizer, loss=loss_function, metrics=addition_metrics)
model.summary()
history = model.fit(X_train, y_train,epochs=number_of_epochs, verbose=verbosity_mode,batch_size=128,validation_data=(X_test,y_test),callbacks=callbacks)
test_results = model.evaluate(cv_test_reviews, test_sentiments, verbose=False)

# print(f'Test results - Loss: {test_results[0]} - Accuracy:{100*test_results[1]}')

(50000,)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_16 (Dropout)         (None, 20000)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                1280064   
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
313/313 [==============================] - 5s 15ms/step - loss: 0.5289 - accuracy: 0.5020 - val_loss: 0.2393 - val_accuracy: 0.4949
Epoch 2/100
313/313 [==============================] - 4s 11ms/step - loss: 0.2211 - accu

ValueError: in user code:

    /Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josiah/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
  

Logistic Regression: Bag of words and Term Frequency-Inverse document Frequency


In [ ]:
print(test_results)

NameError: name 'test_results' is not defined

In [ ]:
lr = LogisticRegression(penalty='l2',max_iter=200, verbose=1, random_state=77)
logistic_bow= lr.fit(cv_train_reviews, train_sentiments)
print(logistic_bow)
logistic_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(logistic_tfidf)



In [ ]:
logistic_bow_pred = lr.predict(cv_test_reviews)
print(logistic_bow_pred)
logistic_tfidf_pred=lr.predict(tv_test_reviews)
print(logistic_tfidf_pred)


In [ ]:
lr_bow_report=classification_report(test_sentiments,logistic_bow_pred, target_names=['Positive','Negative'])
print(lr_bow_report)
lr_tfidf_report=classification_report(test_sentiments, logistic_tfidf_pred, target_names=['Positive','Negative'])
print(lr_tfidf_report)

In [ ]:
lr_bow_report=classification_report(test_sentiments,logistic_bow_pred, target_names=['Positive','Negative'])
print(lr_bow_report)

lr_tfidf_report=classification_report(test_sentiments, logistic_tfidf_pred, target_names=['Positive', 'Negative'])
print(lr_tfidf_report)


In [ ]:
svm=SGDClassifier(loss='hinge', max_iter=1000, random_state=77)
svm_bow=svm.fit(cv_train_reviews, train_sentiments)
print(svm_bow)
svm_tfidf=svm.fit(tv_train_reviews, train_sentiments)
print(svm_tfidf)





In [ ]:

svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

svm_bow_report=classification_report(test_sentiments, svm_bow_predict, target_names=['Positive','Negative'])
print(svm_bow_report)
svm_tfidf_report=classification_report(test_sentiments, svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)


In [ ]:
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

In [ ]:
#Classification report for bag of words 
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Classification report for tfidf features
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

Multinomial Naive Bayes

In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

In [ ]:
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

Use Word Cloud to show the most popular positive and negative reviews

In [ ]:
plt.figure(figsize=(10,10))
positive_text=train_reviews[1]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show


In [ ]:
plt.figure(figsize=(10,10))
negative_text=train_reviews[8]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
negative_words=WC.generate(negative_text)
plt.imshow(negative_words,interpolation='bilinear')
plt.show